In [1]:
import os
import time

import torch
import shutil
import numpy as np
import torch.nn as nn

from pathlib import Path
import pytorch_lightning as pl
from dataclasses import dataclass
from pytorch_lightning import Trainer
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

# Utils

In [2]:
from fimodemix import results_path

class ExperimentsFiles:
    """
    """
    def __init__(self,experiment_dir=None,experiment_indentifier=None,delete=False):
        self.delete = delete
        self.define_experiment_folder(experiment_dir,experiment_indentifier)
        self.create_directories()

    def define_experiment_folder(self,experiment_dir=None,experiment_indentifier=None):
        if experiment_dir is None:
            results_dir = str(results_path)
            if experiment_indentifier is None:
                experiment_indentifier = str(int(time.time()))
            self.experiment_dir = os.path.join(results_dir, experiment_indentifier)        
        self.tensorboard_dir = os.path.join(self.experiment_dir, "logs")
        self.checkpoints_dir = os.path.join(self.experiment_dir, "checkpoints")
    
    def create_directories(self):
        if not Path(self.experiment_dir).exists():
            os.makedirs(self.experiment_dir)
        else:
            if self.delete:
                shutil.rmtree(self.experiment_dir)
                os.makedirs(self.experiment_dir)
            else:
                raise Exception("Folder Exist no Experiments Created Set Delete to True")
            
        if not os.path.isdir(self.tensorboard_dir):
            os.makedirs(self.tensorboard_dir)
        
        if not os.path.isdir(self.checkpoints_dir):
            os.makedirs(self.checkpoints_dir)

# Parameters

In [3]:
@dataclass
class ModelParams:
    #model files
    experiment_indentifier:str = "test"

    #data
    data_name:str = "lorenz63_random_parameters"
    data_dir:str = None

    batch_size: int = 32
    seq_length: int = 10

    #model
    input_size: int = 1  # Original input size
    n_heads: int = 4
    hidden_dim: int = 64
    output_size: int = 1

    #training
    num_epochs: int = 10
    learning_rate: float = 0.001
    embed_dim: int = 8  # New embedding dimension

# Sample Model

In [4]:
from matplotlib import pyplot as plt

# Function to plot paths
def plot_lorenz_paths(paths):
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')

    for i, path in enumerate(paths):
        ax.plot(path[:, 0].numpy(), path[:, 1].numpy(), path[:, 2].numpy(), label=f'Path {i + 1}')

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title('Lorenz 63 Model Paths with Diffusion')
    ax.legend()
    plt.show()

# Data Loader

In [5]:
class TimeSeriesDataset(Dataset):
    def __init__(self, data, targets, seq_length):
        self.X = []
        self.y = []
        for i in range(len(data) - seq_length):
            self.X.append(data[i:i + seq_length])
            self.y.append(targets[i + seq_length])
        self.X = np.array(self.X)
        self.y = np.array(self.y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return torch.FloatTensor(self.X[idx]), torch.FloatTensor([self.y[idx]])

# Model

In [6]:
class TimeSeriesTransformer(pl.LightningModule):

    def __init__(self, input_size, n_heads, hidden_dim, output_size, embed_dim):
        super(TimeSeriesTransformer, self).__init__()

        self.embedding = nn.Linear(input_size, embed_dim)  # Transform input to embed_dim
        assert embed_dim % n_heads == 0, "embed_dim must be divisible by n_heads"
        
        self.attention = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=n_heads)
        self.fc1 = nn.Linear(embed_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_size)
        self.relu = nn.ReLU()
        self.criterion = nn.MSELoss()

    def forward(self, x):
        x = self.embedding(x)  # Transform to (batch_size, seq_length, embed_dim)
        x = x.permute(1, 0, 2)  # Change to (seq_length, batch_size, embed_dim)
        
        attn_output, _ = self.attention(x, x, x)
        x = attn_output.permute(1, 0, 2)  # Back to (batch_size, seq_length, embed_dim)
        x = x.mean(dim=1)  # Global average pooling

        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

    def training_step(self, batch, batch_idx):
        inputs, targets = batch
        outputs = self.forward(inputs)
        loss = self.criterion(outputs, targets)
        self.log('train_loss', loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

# Run Code

In [7]:
# Sample data generation
def generate_data(num_samples=1000):
    np.random.seed(0)
    data = np.random.rand(num_samples, 1)  # 1000 time steps
    targets = np.random.rand(num_samples)   # Corresponding targets
    return data, targets

data, targets = generate_data()

# experiment
experiment_files = ExperimentsFiles()

# Create dataset and DataLoader using ModelParams
params = ModelParams(seq_length=10, batch_size=32)
dataset = TimeSeriesDataset(data=data, targets=targets, seq_length=params.seq_length)
data_loader = DataLoader(dataset, batch_size=params.batch_size, shuffle=True)

# Set up TensorBoard logger
logger = TensorBoardLogger(experiment_files.tensorboard_dir, 
                           name="time_series_transformer")

# Set up Model Checkpointing
checkpoint_callback = ModelCheckpoint(
    monitor='train_loss',
    dirpath=experiment_files.checkpoints_dir,
    filename='best-checkpoint',
    save_top_k=1,
    mode='min',
    save_weights_only=True,
    every_n_train_steps=100  # Save checkpoint every 100 training steps
)

In [8]:
# Instantiate the model and train
model = TimeSeriesTransformer(
    input_size=params.input_size,
    n_heads=params.n_heads,
    hidden_dim=params.hidden_dim,
    output_size=params.output_size,
    embed_dim=params.embed_dim
)

trainer = Trainer(
    max_epochs=params.num_epochs,
    logger=logger,
    callbacks=[checkpoint_callback]
)

trainer.fit(model, data_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type               | Params | Mode 
---------------------------------------------------------
0 | embedding | Linear             | 16     | train
1 | attention | MultiheadAttention | 288    | train
2 | fc1       | Linear             | 576    | train
3 | fc2       | Linear             | 65     | train
4 | relu      | ReLU               | 0      | train
5 | criterion | MSELoss            | 0      | train
--------------------------------------------

Epoch 9: 100%|██████████| 31/31 [00:00<00:00, 173.12it/s, v_num=0]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 31/31 [00:00<00:00, 172.15it/s, v_num=0]
